# Import Libraries 

In [1]:
import pandas as pd
import numpy as np
import re 
import os
import time

In [2]:
import dask.dataframe as dd
from sklearn import preprocessing
import warnings, sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [3]:
import gc
gc.enable()

# Dispaly Settings

In [4]:
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
sns.set(style="whitegrid", color_codes=True)
import matplotlib.cm as cm
%matplotlib inline

pd.set_option('max_colwidth',999)
pd.set_option('display.max_columns', 999)
pd.set_option("display.max_rows",999)

# Vowpal- Wabbit

In [9]:
%%script false
! pip install vowpalwabbit

Couldn't find program: 'false'


In [5]:
%%script false
from sklearn import datasets
from sklearn.model_selection import train_test_split
import vowpalwabbit as vw

Couldn't find program: 'false'


In [7]:
%%script false
X,Y = datasets.make_hastie_10_2(n_samples=10000,random_state=1)
X   = X.astype(np.float32)
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=256)
model = vw.sklearn_vw.VWClassifier()
model.fit(X_train,Y_train)

Couldn't find program: 'false'


In [8]:
%%script false
Y_pred = model.predict(X_test)
model.score(X_train,Y_train)
model.score(X_test,Y_test)

Couldn't find program: 'false'


# ||rl Processing Experiment

Didin't work wil try again on a linux machine as see if it works there

# Helper Functions

In [5]:
def calculateLiftDiscrete(variableList,Target,variableListColumn):
    df = pd.DataFrame()
    i=0
    for variable in variableList:
        
        total            = variable.shape[0]
        totalFrauds      = Target.value_counts(ascending=False)[1]
        totalCleans      = Target.value_counts(ascending=False)[0]
        table            = variable.value_counts(ascending=False).sort_index()
        propTable        = (variable.value_counts(ascending=False)/variable.value_counts(ascending=False).sum()).sort_index()
        crossTab         = pd.crosstab(variable,Target).sort_values(0,ascending=False).sort_index()
        margineFraudGain = round(crossTab[1]/table,4).sort_index()
        detectionRate    = round(crossTab[1]/totalFrauds,4).sort_index()
        sumCleans        = (table-crossTab[1]).sort_index()
        liftRatio        = (round((crossTab[1]/totalFrauds)/(sumCleans/totalCleans),2)).sort_index()
        tempDF           = pd.DataFrame({
                                       'variableName'     : np.repeat(variableListColumn[i],table.count()),
                                       'category'         : table.index,
                                       'totalTrx'         : table.values,
                                       'totalTrxPerCat'   : propTable.values,
                                       'totalFraud'       : crossTab[1].values,
                                       'totalClean'       : crossTab[0].values,
                                       'margineFraudGain' : margineFraudGain.values,
                                       'detectionRate'    : detectionRate.values,
                                       'lift'             : liftRatio.values

                               })
        df = df.append(tempDF,ignore_index=True)
        i=i+1
    return df

In [6]:
def convert_types(df):
    # Convert data types to reduce memory
    for c in df.columns:
        col_type = str(df[c].dtypes)
        numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
        
        # Convert objects to category
        if col_type == 'object':
            df[c] = df[c].astype('category')
        
        # numerics
        elif col_type in numerics:
            c_min = df[c].min()
            c_max = df[c].max()
            if col_type[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[c] = df[c].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[c] = df[c].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[c] = df[c].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[c] = df[c].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[c] = df[c].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[c] = df[c].astype(np.float32)
                else:
                    df[c] = df[c].astype(np.float64)  
        
    return df

In [7]:
def plot_corr_(df,size=10):
    '''Plot a graphical correlation matrix for a dataframe.

    Input:
        df: pandas DataFrame
        size: vertical and horizontal size of the plot'''
    
    
    import matplotlib.pyplot as plt

    # Compute the correlation matrix for the received dataframe
    corr = df.corr()
    
    # Plot the correlation matrix
    fig, ax = plt.subplots(figsize=(size, size))
    cax = ax.matshow(corr, cmap='coolwarm')
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90);
    plt.yticks(range(len(corr.columns)), corr.columns);
    
    # Add the colorbar legend
    cbar = fig.colorbar(cax, ticks=[-1, 0, 1], aspect=40, shrink=.8)



In [8]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
    """Generate a simple plot of the test and training learning curve"""
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes,scoring="roc_auc",shuffle=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [9]:
def basicStats(df,Target):
    stats = []
    [
        stats.append(
            (col, df[col].nunique(),
             df[col].isnull().sum() * 100 / df.shape[0],
             df[col].value_counts(normalize=True, dropna=False).values[0] *
             100,
             df[col].corr(df[Target]) if col in ['float16','int8','float32','int16'] else 0 ,
             np.var(df[col]) if col in ['float16','int8','float32','int16'] else df[col].nunique(),
             (df[col].nunique()/df.shape[0])*100,
             df[col].dtype)) for col in df.columns.tolist()
    ]
    return pd.DataFrame(stats,
                        columns=[
                            'Feature', 'Unique_values',
                            'Percentage of missing values',
                            'Percentage of values in the biggest category',
                            'Cor-relation with Target',
                            'Zero-Variance Check',
                            '%_Cardinality',
                            'type'
                        ]).sort_values(by = ['Percentage of missing values','Zero-Variance Check','%_Cardinality'],
                                       ascending=False)

# Problem Statement 

The goal of this competition is to predict a Windows machine’s probability of getting infected by various families of malware, based on different properties of that machine. The telemetry data containing these properties and the machine infections was generated by combining heartbeat and threat reports collected by Microsoft's endpoint protection solution, Windows Defender.

Each row in this dataset corresponds to a machine, uniquely identified by a MachineIdentifier. HasDetections is the ground truth and indicates that Malware was detected on the machine. Using the information and labels in train.csv, you must predict the value for HasDetections for each machine in test.csv.

The sampling methodology used to create this dataset was designed to meet certain business constraints, both in regards to user privacy as well as the time period during which the machine was running. Malware detection is inherently a time-series problem, but it is made complicated by the introduction of new machines, machines that come online and offline, machines that receive patches, machines that receive new operating systems, etc. While the dataset provided here has been roughly split by time, the complications and sampling requirements mentioned above may mean you may see imperfect agreement between your cross validation, public, and private scores! Additionally, this dataset is not representative of Microsoft customers’ machines in the wild; it has been sampled to include a much larger proportion of malware machines.

# Load Data 

### To Consume less Memory : Define Data Types

In [10]:
dtypes = {
    'MachineIdentifier': 'category',
    'ProductName': 'category',
    'EngineVersion': 'category',
    'AppVersion': 'category',
    'AvSigVersion': 'category',
    'IsBeta': 'int8',
    'RtpStateBitfield': 'float16',
    'IsSxsPassiveMode': 'int8',
    'DefaultBrowsersIdentifier': 'float16',
    'AVProductStatesIdentifier': 'float32',
    'AVProductsInstalled': 'float16',
    'AVProductsEnabled': 'float16',
    'HasTpm': 'int8',
    'CountryIdentifier': 'int16',
    'CityIdentifier': 'float32',
    'OrganizationIdentifier': 'float16',
    'GeoNameIdentifier': 'float16',
    'LocaleEnglishNameIdentifier': 'int8',
    'Platform': 'category',
    'Processor': 'category',
    'OsVer': 'category',
    'OsBuild': 'int16',
    'OsSuite': 'int16',
    'OsPlatformSubRelease': 'category',
    'OsBuildLab': 'category',
    'SkuEdition': 'category',
    'IsProtected': 'float16',
    'AutoSampleOptIn': 'int8',
    'PuaMode': 'category',
    'SMode': 'float16',
    'IeVerIdentifier': 'float16',
    'SmartScreen': 'category',
    'Firewall': 'float16',
    'UacLuaenable': 'float32',
    'Census_MDC2FormFactor': 'category',
    'Census_DeviceFamily': 'category',
    'Census_OEMNameIdentifier': 'float16',
    'Census_OEMModelIdentifier': 'float32',
    'Census_ProcessorCoreCount': 'float16',
    'Census_ProcessorManufacturerIdentifier': 'float16',
    'Census_ProcessorModelIdentifier': 'float16',
    'Census_ProcessorClass': 'category',
    'Census_PrimaryDiskTotalCapacity': 'float32',
    'Census_PrimaryDiskTypeName': 'category',
    'Census_SystemVolumeTotalCapacity': 'float32',
    'Census_HasOpticalDiskDrive': 'int8',
    'Census_TotalPhysicalRAM': 'float32',
    'Census_ChassisTypeName': 'category',
    'Census_InternalPrimaryDiagonalDisplaySizeInInches': 'float16',
    'Census_InternalPrimaryDisplayResolutionHorizontal': 'float16',
    'Census_InternalPrimaryDisplayResolutionVertical': 'float16',
    'Census_PowerPlatformRoleName': 'category',
    'Census_InternalBatteryType': 'category',
    'Census_InternalBatteryNumberOfCharges': 'float32',
    'Census_OSVersion': 'category',
    'Census_OSArchitecture': 'category',
    'Census_OSBranch': 'category',
    'Census_OSBuildNumber': 'int16',
    'Census_OSBuildRevision': 'int32',
    'Census_OSEdition': 'category',
    'Census_OSSkuName': 'category',
    'Census_OSInstallTypeName': 'category',
    'Census_OSInstallLanguageIdentifier': 'float16',
    'Census_OSUILocaleIdentifier': 'int16',
    'Census_OSWUAutoUpdateOptionsName': 'category',
    'Census_IsPortableOperatingSystem': 'int8',
    'Census_GenuineStateName': 'category',
    'Census_ActivationChannel': 'category',
    'Census_IsFlightingInternal': 'float16',
    'Census_IsFlightsDisabled': 'float16',
    'Census_FlightRing': 'category',
    'Census_ThresholdOptIn': 'float16',
    'Census_FirmwareManufacturerIdentifier': 'float16',
    'Census_FirmwareVersionIdentifier': 'float32',
    'Census_IsSecureBootEnabled': 'int8',
    'Census_IsWIMBootEnabled': 'float16',
    'Census_IsVirtualDevice': 'float16',
    'Census_IsTouchEnabled': 'int8',
    'Census_IsPenCapable': 'int8',
    'Census_IsAlwaysOnAlwaysConnectedCapable': 'float16',
    'Wdft_IsGamer': 'float16',
    'Wdft_RegionIdentifier': 'float16',
    'HasDetections': 'int8'
}

### Loading via Dask

In [11]:
train_file_path = "C:/Users/suryadeept/Desktop/CyberSecurity/Malware Detection-Microsoft/train/train.csv"
daskDF          = dd.read_csv(train_file_path,dtype = dtypes)
trainDF         = daskDF.compute()

In [12]:
trainDF.shape

(8921483, 83)

In [13]:
trainDF = convert_types(trainDF)

In [19]:
trainDF.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_ProcessorClass,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,29,128035.0,18.0,35.0,-85,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,NaN,1.0,1.0,Desktop,Windows.Desktop,2668.0,9124.0,4.0,5.0,2340.0,NaN,476940.0,HDD,299451.0,0,4096.0,Desktop,18.906250,1440.0,900.0,Desktop,NaN,4.294967e+09,10.0.17134.165,amd64,rs4_release,17134,165,Professional,PROFESSIONAL,UUPUpgrade,26.0,119,UNKNOWN,0,IS_GENUINE,Retail,NaN,0.0,Retail,NaN,628.0,36144.0,0,NaN,0.0,0,0,0.0,0.0,10.0,0
1,000007535c3f730efa9ea0b7ef1bd645,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,93,1482.0,18.0,119.0,64,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,NaN,1.0,1.0,Notebook,Windows.Desktop,2668.0,91656.0,4.0,5.0,2404.0,NaN,476940.0,HDD,102385.0,0,4096.0,Notebook,13.898438,1366.0,768.0,Mobile,NaN,1.000000e+00,10.0.17134.1,amd64,rs4_release,17134,1,Professional,PROFESSIONAL,IBSClean,8.0,31,UNKNOWN,0,OFFLINE,Retail,NaN,0.0,NOT_SET,NaN,628.0,57858.0,0,NaN,0.0,0,0,0.0,0.0,8.0,0
2,000007905a28d863f6d0d597892cd692,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1341.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,86,153579.0,18.0,64.0,49,windows10,x64,10.0.0.0,17134,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1.0,0,NaN,0.0,137.0,RequireAdmin,1.0,1.0,Desktop,Windows.Desktop,4908.0,317701.0,4.0,5.0,1972.0,NaN,114473.0,SSD,113907.0,0,4096.0,Desktop,21.500000,1920.0,1080.0,Desktop,NaN,4.294967e+09,10.0.17134.165,amd64,rs4_release,17134,165,Core,CORE,UUPUpgrade,7.0,30,FullAuto,0,IS_GENUINE,OEM:NONSLP,NaN,0.0,Retail,NaN,142.0,52682.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0
3,00000b11598a75ea8ba1beea8459149f,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1527.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,88,20710.0,NaN,117.0,115,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,ExistsNotSet,1.0,1.0,Desktop,Windows.Desktop,1443.0,275890.0,4.0,5.0,2272.0,NaN,238475.0,UNKNOWN,227116.0,0,4096.0,MiniTower,18.500000,1366.0,768.0,Desktop,NaN,4.294967e+09,10.0.17134.228,amd64,rs4_release,17134,228,P

### Basic Data Stats

In [20]:
basicStats(trainDF,'HasDetections')[:10]

,Feature,Unique_values,Percentage of missing values,Percentage of values in the biggest category,Cor-relation with Target,type
28,PuaMode,2,99.974119,99.974119,0,category
41,Census_ProcessorClass,3,99.589407,99.589407,0,category
8,DefaultBrowsersIdentifier,1730,95.141637,95.141637,0,float16
68,Census_IsFlightingInternal,2,83.044030,83.044030,0,float16
52,Census_InternalBatteryType,78,71.046809,71.046809,0,category
71,Census_ThresholdOptIn,2,63.524472,63.524472,0,float16
75,Census_IsWIMBootEnabled,2,63.439038,63.439038,0,float16
31,SmartScreen,21,35.610795,48.379658,0,category
15,OrganizationIdentifier,49,30.841487,47.037662,0,float16
29,SMode,2,6.027686,93.928812,0,float16


- based on the above stats I chose to drop top three variable with highest number of missing values associated to it

In [22]:
trainDF.drop(columns = basicStats(trainDF,'HasDetections')['Feature'][:3].tolist(),inplace=True)

In [24]:
trainDF.shape

(8921483, 80)

### Zero-Variance Test 

In [32]:
basicStats(trainDF,'HasDetections').sort_values(by = 'Zero-Variance Check')[:5]

,Feature,Unique_values,Percentage of missing values,Percentage of values in the biggest category,Cor-relation with Target,Zero-Variance Check,type
65,Census_IsFlightingInternal,2,83.04403,83.044030,0,2,float16
42,Census_HasOpticalDiskDrive,2,0.00000,92.281272,0,2,int8
26,AutoSampleOptIn,2,0.00000,99.997108,0,2,int8
11,HasTpm,2,0.00000,98.797106,0,2,int8
7,IsSxsPassiveMode,2,0.00000,98.266622,0,2,int8


- No columns with Zero-variance

### High Cardinality Check

In [42]:
basicStats(trainDF, 'HasDetections').sort_values(by='%_Cardinality',ascending=False)[:5]

,Feature,Unique_values,Percentage of missing values,Percentage of values in the biggest category,Cor-relation with Target,Zero-Variance Check,%_Cardinality,type
0,MachineIdentifier,8921483,0.000000,0.000011,0,8921483,100.000000,category
41,Census_SystemVolumeTotalCapacity,536848,0.594094,0.594094,0,536848,6.017475,float32
35,Census_OEMModelIdentifier,175365,1.145919,3.416271,0,175365,1.965649,float32
13,CityIdentifier,107366,3.647477,3.647477,0,107366,1.203455,float32
70,Census_FirmwareVersionIdentifier,50494,1.794915,1.794915,0,50494,0.565982,float32


- it seems only the MachineIdentifier has very high % of unique values 

### Duplicate Rows

In [111]:
trainDF[trainDF.duplicated()].shape

(0, 80)

- No duplicate COlumns present

# Pre-processing

## Separating the DataFrame based on Type 

### Categorical Columns 

In [100]:
trainCatDF = trainDF.select_dtypes(include=['category']).copy()

In [101]:
trainCatDF['HasDetections'] = trainDF['HasDetections'].copy()

In [102]:
trainCatDF.shape

(8921483, 29)

In [28]:
trainCatDF.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,Platform,Processor,OsVer,OsPlatformSubRelease,OsBuildLab,SkuEdition,SmartScreen,Census_MDC2FormFactor,Census_DeviceFamily,Census_PrimaryDiskTypeName,Census_ChassisTypeName,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSWUAutoUpdateOptionsName,Census_GenuineStateName,Census_ActivationChannel,Census_FlightRing,HasDetections
0,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,windows10,x64,10.0.0.0,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,NaN,Desktop,Windows.Desktop,HDD,Desktop,Desktop,NaN,10.0.17134.165,amd64,rs4_release,Professional,PROFESSIONAL,UUPUpgrade,UNKNOWN,IS_GENUINE,Retail,Retail,0
1,000007535c3f730efa9ea0b7ef1bd645,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,windows10,x64,10.0.0.0,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,NaN,Notebook,Windows.Desktop,HDD,Notebook,Mobile,NaN,10.0.17134.1,amd64,rs4_release,Professional,PROFESSIONAL,IBSClean,UNKNOWN,OFFLINE,Retail,NOT_SET,0
2,000007905a28d863f6d0d597892cd692,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1341.0,windows10,x64,10.0.0.0,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,RequireAdmin,Desktop,Windows.Desktop,SSD,Desktop,Desktop,NaN,10.0.17134.165,amd64,rs4_release,Core,CORE,UUPUpgrade,FullAuto,IS_GENUINE,OEM:NONSLP,Retail,0
3,00000b11598a75ea8ba1beea8459149f,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1527.0,windows10,x64,10.0.0.0,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,ExistsNotSet,Desktop,Windows.Desktop,UNKNOWN,MiniTower,Desktop,NaN,10.0.17134.228,amd64,rs4_release,Professional,PROFESSIONAL,UUPUpgrade,FullAuto,IS_GENUINE,OEM:NONSLP,Retail,1
4,000014a5f00daa18e76b81417eeb99fc,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1379.0,windows10,x64,10.0.0.0,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,RequireAdmin,Notebook,Windows.Desktop,HDD,Portable,Mobile,lion,10.0.17134.191,amd64,rs4_release,Core,CORE,Update,FullAuto,IS_GENUINE,Retail,Retail,1


#### Null Check

In [103]:
((trainCatDF.isnull().sum()/trainCatDF.shape[0])*100).sort_values(ascending = False)[:6]

Census_InternalBatteryType      71.046809
SmartScreen                     35.610795
Census_PrimaryDiskTypeName       0.143967
Census_ChassisTypeName           0.006983
Census_PowerPlatformRoleName     0.000616
OsBuildLab                       0.000235
dtype: float64

In [30]:
((trainCatDF.isnull().sum() / trainCatDF.shape[0]) *
            100).sort_values(ascending=False)[:6].index[0]

'Census_InternalBatteryType'

In [77]:
pd.crosstab(
    trainCatDF[((trainCatDF.isnull().sum() / trainCatDF.shape[0]) *
                100).sort_values(ascending=False)
               [:6].index[2]].cat.add_categories('MISS').fillna('MISS'),
    trainCatDF.HasDetections).sort_values(1,ascending=False)[:6]

HasDetections,0,1
Census_PrimaryDiskTypeName,,
HDD,2865755,2941049
SSD,1240186,1226622
UNKNOWN,197160,161091
Unspecified,152835,123941
MISS,6655,6189


In [35]:
pd.crosstab(
    trainCatDF[((trainCatDF.isnull().sum() / trainCatDF.shape[0]) *
                100).sort_values(ascending=False)
               [:6].index[0]].cat.add_categories('MISS').fillna('MISS'),
    trainCatDF.HasDetections).sort_values(1,ascending=False)[:5]

HasDetections,0,1
Census_InternalBatteryType,,
MISS,3136146,3202283
lion,1048882,979374
li-i,126132,119485
#,88815,95183
lip,29100,32999


In [36]:
pd.crosstab(
    trainCatDF[((trainCatDF.isnull().sum() / trainCatDF.shape[0]) *
                100).sort_values(ascending=False)
               [:6].index[1]].cat.add_categories('MISS').fillna('MISS'),
    trainCatDF.HasDetections).sort_values(1,ascending=False)[:5]

HasDetections,0,1
SmartScreen,,
RequireAdmin,2428260,1887923
MISS,1650637,1526374
ExistsNotSet,201533,844650
Off,94594,91959
Warn,57382,78101


- OSBuildLab, Census_PowerPlatformRoleName  and Census_ChassisTypeName have statistically insignificant % of null values thus, I will drop these missing values.
- Census_InternalBatteryType ,Census_PrimaryDiskTypeName & SmartScreen has considerable amount of null values associated to them. However, they have corresponding number for cleans as well thus, I will replace them as '_MISS' 

In [104]:
trainCatDF['Census_InternalBatteryType_MISS_Handled'] = np.where(
    trainCatDF['Census_InternalBatteryType'].isnull(), '_MISS',
    trainCatDF['Census_InternalBatteryType'])

In [105]:
trainCatDF['SmartScreene_MISS_Handled'] = np.where(
    trainCatDF['SmartScreen'].isnull(), '_MISS',
    trainCatDF['SmartScreen'])

In [106]:
trainCatDF['Census_PrimaryDiskTypeName_MISS_Handled'] = np.where(
    trainCatDF['Census_PrimaryDiskTypeName'].isnull(), '_MISS',
    trainCatDF['Census_PrimaryDiskTypeName'])

In [107]:
trainCatDF.drop(columns=['Census_InternalBatteryType',
                         'SmartScreen',
                         'Census_PrimaryDiskTypeName'
                        ],inplace=True)

In [109]:
trainCatDF.dropna(inplace=True)

In [110]:
((trainCatDF.isnull().sum() / trainCatDF.shape[0]) *
            100).sort_values(ascending=False)[:6]

Census_PrimaryDiskTypeName_MISS_Handled    0.0
Census_ChassisTypeName                     0.0
ProductName                                0.0
EngineVersion                              0.0
AppVersion                                 0.0
AvSigVersion                               0.0
dtype: float64

#### Numerical Columns